In [1]:
# required Python imports
import numpy as np
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.3.07 imported


## Step 1: Create the database

Questions to be investigated:

- How many measurements are required to obtain the orientation with an error of, say, ±1 degree?
- What range of azimuths should we consider? 90, 180, 360? Does it matter?
- Are there special cases leaving aside the gymbal lock issue (see below)?
- Other considerations?

> ### An important note about the gymbal lock issue
>
> A special case when using Euler angles is the so-called gymbal lock. This occurs when the second Euler angle is at one of its limits (0, 360, or 180 in the case of an orthorhombic system). In this case, the axes of rotation of the first and last Euler rotations are either the same or exactly opposite, i.e. an infinite number of angles $\phi1$ and $\phi2$ will represent the same rotation even if we restrict the range to an interval of length 2π: infinite pairs of $\phi1$ and $\phi2$ angles satisfy
>
> $$
> \phi1 + \phi2 = \text{constant} \quad | \quad \phi1 - \phi2 = \text{constant}
> $$
>
> For example, all these triplets will represent the same orientation (all representing a 45 degree rotation along the z-axis):
>
> $[45, 0, 0], [30, 0, 15], [50, 0, -5], [0, 0, 45], [55, 0, 170]...\text{and so on} $

### 1.1 Euler angles

In [2]:
# check explore Euler space avoiding Phi=0
f.explore_Euler_space(step=1, lower_bounds=(0, 2, 0))

array([[  0,   2,   0],
       [  0,   2,   1],
       [  0,   2,   2],
       ...,
       [ 90,  90, 178],
       [ 90,  90, 179],
       [ 90,  90, 180]])

In [3]:
# STEP 1.1: generate Euler angles and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=29, lower_bounds=(0, 3, 0)))
database.euler.shape

(112, 3)

### 1.2 Generate reference T envelope

In [4]:
# STEP 1.2: Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(90, 50, 20), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

In [5]:
# example consider the full range of azimuths and 16 measures
np.arange(0, 360, 45/2)

array([  0. ,  22.5,  45. ,  67.5,  90. , 112.5, 135. , 157.5, 180. ,
       202.5, 225. , 247.5, 270. , 292.5, 315. , 337.5])

In [6]:
# example consider the a 180 degrees range and 16 measures
np.arange(0, 180, 45/4)

array([  0.  ,  11.25,  22.5 ,  33.75,  45.  ,  56.25,  67.5 ,  78.75,
        90.  , 101.25, 112.5 , 123.75, 135.  , 146.25, 157.5 , 168.75])

### 1.3 Calculate and store in the database the T values and azimuths of sections

In [7]:
# STEP 1.3: Generate 16 measures at different azimuth angles
angles = np.arange(0, 360, 45/2)

# initialize variables
T_vals = []
azi_vals = []

for euler in database.euler:
    # rotate
    x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

    # extract XY intersection
    xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

    # get the indexes of specific angles
    indexes = f.find_nearest(xy_vectors['angles'], angles)

    # append values
    T_vals.append(xy_vectors.loc[indexes, ['T']].T.values.tolist()[0])
    azi_vals.append(xy_vectors.loc[indexes, ['angles']].T.values.tolist()[0])

# store values in database
database.T_values = np.array(T_vals)
database.azimuths = np.array(azi_vals)

In [8]:
print(database.euler.shape)
print(database.T_values.shape)
print(database.azimuths.shape)

(112, 3)
(112, 16)
(112, 16)


In [9]:
print('Euler angles: ', database.euler[-1])
print('T values: ', np.around(database.T_values[-1], 1))
print('Azimuths: ', np.around(database.azimuths[-1], 1))

Euler angles:  [ 87  90 174]
T values:  [49.8 43.2 32.  22.4 20.2 26.8 38.1 47.7 49.8 43.2 32.  22.4 20.2 26.8
 38.1 47.7]
Azimuths:  [  0.1  22.6  44.9  67.7  89.1 112.4 134.9 157.6 180.1 202.6 224.9 247.7
 269.1 292.4 314.9 337.6]


In [10]:
# for i in range(45):
#     print(np.around(database.azimuths[i], 1), '; ', np.around(database.euler[i], 0))

## 2. Test minimization algorithms

In [11]:
for index, orientation in enumerate(database.euler):
    print('Real:', np.around(orientation, 0))

    measures = np.column_stack((database.T_values[index],
                                database.azimuths[index],
                                np.full_like(database.azimuths[index], 90)))

    print('default algorithm')
    f.find_orientation(measurements=measures, params=(90, 50, 20))
    
    print('differential evolution algorithm')
    f.find_orientation_diffevol(measurements=measures, params=(90, 50, 20))

    print('dual annealing algorithm')
    f.find_orientation_annealing(measurements=measures, params=(90, 50, 20))
    print('----------------------')

Real: [0 3 0]
default algorithm
Calculated orientation: [0. 3. 0.]
differential evolution algorithm
Calculated Orientation: [  3.   3. 177.]
dual annealing algorithm
Calculated Orientation: [0. 3. 0.]
----------------------
Real: [ 0  3 29]
default algorithm
Calculated orientation: [ 3.  3. 26.]
differential evolution algorithm
Calculated Orientation: [ 29.   3. 180.]
dual annealing algorithm
Calculated Orientation: [ 61.   1. 148.]
----------------------
Real: [ 0  3 58]
default algorithm
Calculated orientation: [ 3.  3. 55.]
differential evolution algorithm
Calculated Orientation: [ 3.  3. 55.]
dual annealing algorithm
Calculated Orientation: [ 9.  3. 49.]
----------------------
Real: [ 0  3 87]
default algorithm
Calculated orientation: [ 2.  3. 85.]
differential evolution algorithm
Calculated Orientation: [ 2.  3. 85.]
dual annealing algorithm
Calculated Orientation: [ 88.   1. 179.]
----------------------
Real: [  0   3 116]
default algorithm
Calculated orientation: [ 11.   3. 105.

TODO:
- Carry out a test comparing the different methods at the same time (time, accuracy, etc.).
- See if there are any special situations, e.g. when Phi=0 is one case.
- The question of range of azimuths and number of points vs. accuracy.
- See how to deal with equivalent orientations, e.g. [60, 0, 0, 0] == [30, 0, 30] and so on.


In [13]:
import sys
from datetime import date    
today = date.today().isoformat()

print(f'Notebook tested in {today} using:')
print('Python', sys.version)
print('Numpy', np.__version__)

Notebook tested in 2024-03-07 using:
Python 3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:15:57) [MSC v.1916 64 bit (AMD64)]
Numpy 1.26.4
